In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import datetime
import os
import requests
import json
from tqdm import tqdm
import geopandas as gpd
import numpy as np
import fnmatch

In [11]:
# Define paths
data = './data'
supply = os.path.join(data, 'original_data', 'supply_related')
demand = os.path.join(data, 'original_data', 'demand_related')

In [3]:
from_date = '2020-05-01'
to_date = '2021-12-31'

start_date = datetime.datetime.strptime(from_date,  "%Y-%m-%d")
end_date = datetime.datetime.strptime(to_date,  "%Y-%m-%d")

focus_date = []
focus_date_death = []
delta = datetime.timedelta(days=1)
while start_date <= end_date:
    focus_date.append(start_date.strftime("%Y-%m-%d"))
    focus_date_death.append(start_date.strftime("%m/%d/%Y"))
    start_date += delta
    

['05/01/2020',
 '05/02/2020',
 '05/03/2020',
 '05/04/2020',
 '05/05/2020',
 '05/06/2020',
 '05/07/2020',
 '05/08/2020',
 '05/09/2020',
 '05/10/2020',
 '05/11/2020',
 '05/12/2020',
 '05/13/2020',
 '05/14/2020',
 '05/15/2020',
 '05/16/2020',
 '05/17/2020',
 '05/18/2020',
 '05/19/2020',
 '05/20/2020',
 '05/21/2020',
 '05/22/2020',
 '05/23/2020',
 '05/24/2020',
 '05/25/2020',
 '05/26/2020',
 '05/27/2020',
 '05/28/2020',
 '05/29/2020',
 '05/30/2020',
 '05/31/2020',
 '06/01/2020',
 '06/02/2020',
 '06/03/2020',
 '06/04/2020',
 '06/05/2020',
 '06/06/2020',
 '06/07/2020',
 '06/08/2020',
 '06/09/2020',
 '06/10/2020',
 '06/11/2020',
 '06/12/2020',
 '06/13/2020',
 '06/14/2020',
 '06/15/2020',
 '06/16/2020',
 '06/17/2020',
 '06/18/2020',
 '06/19/2020',
 '06/20/2020',
 '06/21/2020',
 '06/22/2020',
 '06/23/2020',
 '06/24/2020',
 '06/25/2020',
 '06/26/2020',
 '06/27/2020',
 '06/28/2020',
 '06/29/2020',
 '06/30/2020',
 '07/01/2020',
 '07/02/2020',
 '07/03/2020',
 '07/04/2020',
 '07/05/2020',
 '07/06/20

## Daily Confirmed Case

In [4]:
confirm_case = pd.read_excel(os.path.join(demand, 'Texas COVID-19 New Confirmed Cases by County.xlsx'),
                             sheet_name='New Cases by County 2020',
                             header=2,
                             index_col=0)

confirm_case = confirm_case.loc['Anderson':'Zavala', focus_date]
confirm_case.columns = confirm_case.columns.astype(str)

# Remove negative value of confirmed cases. 
# Negative value has replaced with 0
for col in confirm_case.columns:
    confirm_case[col] = confirm_case[col].mask(confirm_case[col] < 0, 0)

rename_lookup = dict(zip(focus_date, focus_date_death))
confirm_case = confirm_case.rename(columns=rename_lookup, errors='raise')
    
confirm_case

,05/01/2020,05/02/2020,05/03/2020,05/04/2020,05/05/2020,05/06/2020,05/07/2020,05/08/2020,05/09/2020,05/10/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
County,,,,,,,,,,,,,,,,,,,,,
Anderson,5.0,2.0,0.0,15.0,14.0,13.0,42.0,12.0,8.0,5.0,...,1.0,6.0,2.0,7.0,3.0,7.0,4.0,13.0,4.0,4.0
Andrews,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,10.0,5.0,0.0,0.0,0.0,0.0,16.0,27.0,16.0
Angelina,1.0,1.0,0.0,4.0,11.0,1.0,3.0,2.0,0.0,0.0,...,1.0,7.0,6.0,6.0,7.0,9.0,9.0,11.0,7.0,23.0
Aransas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,5.0,2.0,1.0,9.0,0.0,1.0,4.0
Archer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,6.0,3.0,2.0,2.0,3.0,11.0,3.0,2.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wood,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.0,3.0,2.0,8.0,1.0,9.0,1.0,6.0,1.0,2.0
Yoakum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
Young,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,8.0,1.0,1.0,3.0


## Calculate Daily Death from COVID-19

In [5]:
# Loaded data is the accumulated death per county
death_case = pd.read_excel(os.path.join(demand, 'Texas COVID-19 Fatality Count Data by County.xlsx'),
                             sheet_name='Fatalities by County 2020',
                             header=2,
                             index_col=0)

death_case = death_case.loc['Anderson':'Zavala', focus_date_death]

death_case

,05/01/2020,05/02/2020,05/03/2020,05/04/2020,05/05/2020,05/06/2020,05/07/2020,05/08/2020,05/09/2020,05/10/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
County,,,,,,,,,,,,,,,,,,,,,
Anderson,1,1,1,1,1,1,1,1,1,1,...,209,210,210,210,211,211,212,212,212,212
Andrews,0,0,0,0,0,0,0,0,0,0,...,66,66,66,66,66,66,67,67,67,67
Angelina,2,2,2,2,2,2,2,3,3,3,...,430,430,430,430,430,430,430,431,431,431
Aransas,0,0,0,0,0,0,0,0,0,0,...,68,68,68,68,68,68,68,68,68,68
Archer,0,0,0,0,0,0,0,0,0,0,...,24,24,24,24,24,24,24,24,24,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wood,0,0,0,0,0,0,0,0,0,0,...,242,242,243,243,243,243,244,244,245,245
Yoakum,0,0,0,0,0,0,0,0,0,0,...,35,35,35,35,35,35,35,35,35,36
Young,0,0,0,0,0,0,0,0,0,0,...,74,74,74,74,74,74,74,74,74,74


In [6]:
# Calculate daily death from the accumulated value
daily_death = pd.DataFrame(index=death_case.index, columns=focus_date_death[1:])

for i, date in enumerate(focus_date_death):
    if i != 0:
        daily_death[date] = death_case[date] - death_case[focus_date_death[i -1]]

# Remove negative value of daily death. 
# Negative value has replaced with 0
for col in daily_death.columns:
    daily_death[col] = daily_death[col].mask(daily_death[col] < 0, 0)
        
daily_death

,05/02/2020,05/03/2020,05/04/2020,05/05/2020,05/06/2020,05/07/2020,05/08/2020,05/09/2020,05/10/2020,05/11/2020,...,12/22/2021,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021
County,,,,,,,,,,,,,,,,,,,,,
Anderson,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
Andrews,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Angelina,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
Aransas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Archer,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wood,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0
Yoakum,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Young,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Accumulate confirmed and death case over the last 7 days

In [7]:
# Make a dictionary that has keys as target date and values as the date that should be averaged. 
focus_date_death_dict = {}
time_delta = list(range(0, -7, -1))

for idx, date in enumerate(focus_date_death):  # Starts from 06/01/2020
    temp_list = []
    for delta in time_delta:
        temp_list.append(
            str(
                (datetime.datetime.strptime(focus_date_death[idx], "%m/%d/%Y") + datetime.timedelta(days=delta)
                ).strftime("%m/%d/%Y"))
        )
        
    focus_date_death_dict[date] = temp_list

# Remove days in May from the dictionary, given that it is not the focus date of our analysis. 
may_date = fnmatch.filter(focus_date_death_dict.keys(), '05/??/2020')

for date in may_date:
    del focus_date_death_dict[date]
    
focus_date_death_dict

{'06/01/2020': ['06/01/2020',
  '05/31/2020',
  '05/30/2020',
  '05/29/2020',
  '05/28/2020',
  '05/27/2020',
  '05/26/2020'],
 '06/02/2020': ['06/02/2020',
  '06/01/2020',
  '05/31/2020',
  '05/30/2020',
  '05/29/2020',
  '05/28/2020',
  '05/27/2020'],
 '06/03/2020': ['06/03/2020',
  '06/02/2020',
  '06/01/2020',
  '05/31/2020',
  '05/30/2020',
  '05/29/2020',
  '05/28/2020'],
 '06/04/2020': ['06/04/2020',
  '06/03/2020',
  '06/02/2020',
  '06/01/2020',
  '05/31/2020',
  '05/30/2020',
  '05/29/2020'],
 '06/05/2020': ['06/05/2020',
  '06/04/2020',
  '06/03/2020',
  '06/02/2020',
  '06/01/2020',
  '05/31/2020',
  '05/30/2020'],
 '06/06/2020': ['06/06/2020',
  '06/05/2020',
  '06/04/2020',
  '06/03/2020',
  '06/02/2020',
  '06/01/2020',
  '05/31/2020'],
 '06/07/2020': ['06/07/2020',
  '06/06/2020',
  '06/05/2020',
  '06/04/2020',
  '06/03/2020',
  '06/02/2020',
  '06/01/2020'],
 '06/08/2020': ['06/08/2020',
  '06/07/2020',
  '06/06/2020',
  '06/05/2020',
  '06/04/2020',
  '06/03/2020',
 

In [9]:
# Add FIPS-County pair to the dataframe
tsa = pd.read_json('./data/original_data/tsa_county.json')
tsa = tsa.set_index('County')
tsa

,FIPS,TSA
County,,
Armstrong,48011,A
Briscoe,48045,A
Carson,48065,A
Childress,48075,A
Collingsworth,48087,A
...,...,...
Ward,48475,J
Winkler,48495,J
Culberson,48109,I


In [21]:
# Number of fatailty per 1000 COVID-19 confirmed cases
accumulate_death = pd.DataFrame(index=daily_death.index, columns=focus_date_death_dict.keys())
accumulate_confirmed_case = pd.DataFrame(index=confirm_case.index, columns=focus_date_death_dict.keys())
seven_day_ratio = pd.DataFrame(index=confirm_case.index, columns=focus_date_death_dict.keys())

for idx, val in focus_date_death_dict.items():
    accumulate_death[idx] = daily_death[val].sum(axis=1)
    accumulate_confirmed_case[idx] = confirm_case[val].sum(axis=1)
    seven_day_ratio[idx] = accumulate_death[idx] / accumulate_confirmed_case[idx] * 1000. # Number of fatailty per 1000 cases

seven_day_ratio.replace(np.inf, np.nan, inplace=True)  # Infinite value was replaced with 0
seven_day_ratio = seven_day_ratio.fillna(0)
seven_day_ratio = seven_day_ratio.merge(tsa, left_index=True, right_index=True)
seven_day_ratio['FIPS'] = seven_day_ratio['FIPS'].astype(str)
seven_day_ratio = seven_day_ratio.set_index('FIPS')
seven_day_ratio

,06/01/2020,06/02/2020,06/03/2020,06/04/2020,06/05/2020,06/06/2020,06/07/2020,06/08/2020,06/09/2020,06/10/2020,...,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021,TSA
FIPS,,,,,,,,,,,,,,,,,,,,,
48001,400.000000,23.809524,24.390244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,50.000000,47.619048,37.037037,74.074074,66.666667,100.000000,71.428571,50.000000,47.619048,G
48003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,43.478261,38.461538,41.666667,43.478261,0.000000,55.555556,32.258065,20.833333,16.949153,J
48005,142.857143,142.857143,333.333333,37.037037,28.571429,26.315789,26.315789,25.641026,29.411765,25.210084,...,83.333333,34.482759,28.571429,23.809524,22.222222,22.222222,18.181818,18.181818,13.888889,H
48007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,U
48009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48499,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,142.857143,125.000000,...,25.000000,54.054054,47.619048,48.780488,41.666667,111.111111,66.666667,107.142857,71.428571,G
48501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000,B
48503,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,C


In [22]:
# Append the geometry to the table
county = gpd.read_file(os.path.join(demand, 'tl_2021_48_county', 'tl_2021_48_county.shp'))
county = county[['GEOID', 'NAME', 'geometry']]
county = county.set_index('GEOID')

county = county.merge(seven_day_ratio, left_index=True, right_index=True)
county

,NAME,geometry,06/01/2020,06/02/2020,06/03/2020,06/04/2020,06/05/2020,06/06/2020,06/07/2020,06/08/2020,...,12/23/2021,12/24/2021,12/25/2021,12/26/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021,TSA
48327,Menard,"POLYGON ((-99.77120 30.71079, -99.78240 30.710...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,K
48189,Hale,"POLYGON ((-101.73504 33.82864, -101.73507 33.8...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,125.000000,...,58.823529,71.428571,80.00000,75.471698,59.701493,60.606061,32.786885,52.631579,104.166667,B
48011,Armstrong,"POLYGON ((-101.08728 35.02708, -101.08729 35.0...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,A
48057,Calhoun,"POLYGON ((-96.35229 28.50953, -96.35241 28.508...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,100.000000,100.000000,100.000000,S
48077,Clay,"POLYGON ((-97.97784 33.81417, -97.97784 33.813...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48205,Hartley,"POLYGON ((-102.31929 36.05545, -102.31679 36.0...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,333.333333,250.00000,250.000000,200.000000,111.111111,83.333333,71.428571,0.000000,A
48249,Jim Wells,"POLYGON ((-97.93395 27.85971, -97.93395 27.859...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,28.571429,U
48505,Zapata,"POLYGON ((-99.15217 27.27142, -99.15088 27.271...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,T
48453,Travis,"POLYGON ((-97.76561 30.43241, -97.76532 30.432...",1166.666667,545.454545,416.666667,12.461059,10.498688,14.814815,12.345679,17.241379,...,0.367377,0.998004,0.81367,1.191327,1.289214,1.227747,1.177394,1.471206,1.030928,O


In [24]:
county.to_file(os.path.join(data, 'processed_data', 'relationship', 'seven_day_ratio.shp'))

/Users/zawoon96/opt/anaconda3/envs/sa/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
